In [1]:
import pandas as pd

In [2]:
top5000_academic = pd.read_csv('CAT_5000_verbs.csv', encoding='utf-8')

In [7]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [3]:
top5000_academic

,lemma,freq
0,мочь,33712
1,являться,26611
2,иметь,18593
3,быть,15181
4,использовать,12889
...,...,...
4995,зам,8
4996,выровнять,8
4997,требуемый,8
4998,достигти,8


In [12]:
really_verbs = []
for verb in top5000_academic['lemma']:
    p = morph.parse(verb)
    is_verb = False
    for option in p:
        if option.tag.POS == 'INFN':
            is_verb = True
            really_verbs.append(verb)
print(len(really_verbs))

4939


In [16]:
import re
with open('Rozental_no_metadata.txt', encoding='utf-8') as f:
    Rozental = f.read()
all_Rozental_verbs = []
government_models = [model.strip() for model in Rozental.split(';')]
for government_model in government_models:
    try:
        [description, verbs] = government_model.split(':')
        verbs = re.sub('[0-9]', '', verbs) #нам не нужны номера значений
        verbs = [verb.strip() for verb in verbs.split(',')]
        for verb in verbs:
            all_Rozental_verbs.append(verb)
    except: ## Если попадутся каке-то абзацы метаданных
        pass

In [17]:
len(all_Rozental_verbs)

3735

In [19]:
need_extra_data = list(set(really_verbs)-set(all_Rozental_verbs))

In [20]:
len(need_extra_data)

3689

In [22]:
collected_government_data = pd.DataFrame({'verbs':need_extra_data})

In [23]:
collected_government_data

,verbs
0,формироваться
1,расширяться
2,отличать
3,выражеть
4,отнимать
...,...
3684,предназначать
3685,однокореть
3686,лишаться
3687,центрировать


In [26]:
import requests
from requests.exceptions import HTTPError
from numpy import median # for printing all fval ranks above the rank median
from decimal import Decimal # for printing freq ratio without truncation

def get_data(word):
    # configure default params to send to the server
    url = 'http://cococo.cosyco.ru/cococo-test.py'
    params = {
        'flimit': "10",
        'word1': word,        # Первое слово (ex. читать)
        'pos1': "",
        'features1': "",
        'word2': "",        # Второе слово (ex. сказать)
        'pos2': "",
        'features2': "",
        'corpus': "RNC",    # Используем ruscorpora
        'flem': "0",
        'alem': "0",
        'skip': "2"         # 2-граммы
    }

    # send get request to the server
    try:
        r = requests.get(url=url, params=params)

        # if server error, raise an exception
        r.raise_for_status()
    except HTTPError as http_err:
        print('HTTP error occurred:', http_err)
    except Exception as err:
        print('Other error occurred:', err)
    else:
        data = r.json()
        return data

In [25]:
def get_cases_freq(word):
    data = get_data(word)
    if data:
        cases = {case: 0 for case in ['gen', 'dat', 'acc', 'ins', 'loc']}
        for pos in data['poses']:
            for feature in pos['features']:
                if feature['fname'] == 'case':
                    for case in feature['fvals']:
                        case_val = case['val']
                        if case_val in cases:
                            cases[case_val] += case['freqs'][0]
        return cases

In [27]:
collected_government_data['bigram_all_cases'] = collected_government_data['verbs'].apply(get_cases_freq)

In [28]:
collected_government_data.to_csv('new_government_data.csv')

In [29]:
def get_preposition_data(verb):
    words_data = get_data(verb)['words']
    if words_data:
        return {word['word']: word['freq1'] for word in words_data}

In [30]:
collected_government_data['preposition_data'] = collected_government_data['verbs'].apply(get_preposition_data)

In [31]:
collected_government_data.to_csv('new_government_data.csv')